In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import numpy as np

In [2]:
game_name_list = []
game_genre_list = []
mean_estimation_list = []
game_platform_list = []

headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 YaBrowser/23.3.4.603 Yowser/2.5 Safari/537.36'
}

for n in range(1,614):
    url = f'https://www.playground.ru/games?release=all&sort=time&p={n}'
    igor = requests.get(url, headers=headers).text
    soup = BS(igor, 'lxml')

    items = soup.find_all('div', class_="info")
    for elem in items:
        # ищем имя
        name = elem.find('div', class_="media-heading title").find('a').text
        game_name_list.append(name)

        # ищем оценку
        try:
            estimation = elem.find('span', class_="value js-game-rating-value").text
            mean_estimation_list.append(float(estimation))
        except AttributeError:
            mean_estimation_list.append(np.NaN)

        # ищем жанры
        geners = elem.find('div', class_="meta-info").find_next().find_next_sibling().find_all('a')
        genre_list = []
        for gener in geners:
            genre_list.append(gener.get_text())
            genre_str = ", ".join(genre_list)
        game_genre_list.append(genre_str)

        # ищем платформы
        platforms = elem.find('div', class_="meta-info").find_next().find_next_sibling().find_next_sibling().find_all('span')
        platform_list = []
        for platform in platforms:
            platform_list.append(platform.get_text())
            platform_str = ", ".join(platform_list)
        game_platform_list.append(platform_str)

game_dt = pd.DataFrame({
    "Title": game_name_list,
    "Genres": game_genre_list,
    "Platforms": game_platform_list,
    "Estimation": mean_estimation_list
})
display(game_dt.head(10))
game_dt.info()

,Title,Genres,Platforms,Estimation
0,Moscow Rush,"Симулятор, Инди, От третьего лица",PC,6.2
1,Asylum,"Адвенчура, Ужасы, От первого лица",PC,8.3
2,GTR 3,"Симулятор, Гонки, Автомобили",PC,8.2
3,SpaceVenture,Адвенчура,PC,7.8
4,The Trench,Экшен,PC,7.1
5,Routine,"Экшен, Адвенчура, Ужасы, Космос",PC,8.5
6,Deep Down,"Ролевая, От третьего лица, Фэнтези, Кооператив...",PS5,7.5
7,TimeSplitters Rewind,"Экшен, Мультиплеер",PC,7.9
8,City of Titans,"Ролевая, Мультиплеер",PC,7.2
9,Terraria 2,"Аркада, Стратегия","PC, PS Vita, PS4, Xbox One",8.5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18390 entries, 0 to 18389
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       18390 non-null  object 
 1   Genres      18390 non-null  object 
 2   Platforms   18390 non-null  object 
 3   Estimation  15720 non-null  float64
dtypes: float64(1), object(3)
memory usage: 574.8+ KB


In [3]:
game_dt.to_excel('data/Playground_game.xlsx', index=False)